In [1]:
import tensorflow as tf
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Dense, concatenate, ReLU,
                                     BatchNormalization, GlobalAveragePooling2D, concatenate)
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, CSVLogger
import os
import keras
import tensorflow
from keras.layers import (Input, Dense, Flatten, Lambda, Dropout, Activation,
        Convolution2D, MaxPooling2D, BatchNormalization, concatenate,  Reshape , GlobalMaxPooling2D, AveragePooling2D)
from keras import backend as K
from keras.models import load_model
from tensorflow.keras.optimizers import Adam 
from sklearn.metrics import confusion_matrix
import seaborn as sns
import os

In [2]:
def process_data_for_conv2D(X, desired_shape=None):
    """
    Reshape the data into the desired size by adding a channel dim.
    It prepares the data for the convolution operations
    
    Paramteres:
    ================================================
    X: ndarray of shape (n_samples, img_width, img_height)
        The input data to be resized
    
    desired_shape: tuple of shape (3,)
        The input of shape attribs
    
    Returns:
    ================================================
    X_conv2D: ndarray of shape (n_sample, img_width, img_height, 1)
        The resized array
    """
    X_conv2D = []
    for sample in X:
        
        # Adding a channel dim to each input array
        sample = np.reshape(sample, newshape=(sample.shape[0], sample.shape[1], 1))
        
        # If there is a desired shape resize the input image to it
        if desired_shape:
            sample = resize(sample, output_shape=desired_shape)
        X_conv2D.append(sample)
        
    # Convert the list into an ndarray
    return np.array(X_conv2D, dtype=np.float32)
def train_val_test_split(X, y, train_size, val_size, test_size):
    X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, train_size=train_size, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=test_size/(test_size + val_size), stratify=y_val_test)
    return X_train, y_train, X_val, y_val, X_test, y_test

In [10]:
'''
Inistintiate the image_loader to load the images from the director
 * Resize images
 * Assign sparse_categorical classes
 * Shuffle the data
'''
img_width, img_height = 646, 128
directory = "../Data/Extended_Ballroom_Processing/"


X_melspec = np.load(directory+"melspec_feature_2048.npy")
y = np.load(directory+"onehot_labels.npy")
num_classes = y.shape[-1]
X_melspec = process_data_for_conv2D(X_melspec)

In [4]:
def broadcast_block(x, filters=32):
    conv_1x1 = BatchNormalization()(x)
    conv_1x1 = ReLU()(conv_1x1)
    conv_1x1 = Conv2D(filters=filters, kernel_size=1, strides=1, padding='same', kernel_initializer="lecun_normal")(conv_1x1)
    
    conv_3x3 = BatchNormalization()(x)
    conv_3x3 = ReLU()(conv_3x3)
    conv_3x3 = Conv2D(filters=filters, kernel_size=1, strides=1, padding='same', kernel_initializer="lecun_normal")(conv_3x3)
    conv_3x3 = BatchNormalization()(conv_3x3)
    conv_3x3 = ReLU()(conv_3x3)
    
    conv_5x5 = BatchNormalization()(x)
    conv_5x5 = ReLU()(conv_5x5)
    conv_5x5 = Conv2D(filters=filters, kernel_size=1, strides=1, padding='same', kernel_initializer="lecun_normal")(conv_5x5)
    conv_5x5 = BatchNormalization()(conv_5x5)
    conv_5x5 = ReLU()(conv_5x5)
    conv_5x5 = Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', kernel_initializer="lecun_normal")(conv_5x5)
    
    max_pool = MaxPooling2D(pool_size=3, strides=1, padding='same')(x)
    max_pool = BatchNormalization()(max_pool)
    max_pool = ReLU()(max_pool)
    max_pool = Conv2D(filters=filters, kernel_size=1, strides=1, padding='same', kernel_initializer="lecun_normal")(max_pool)
    
    out = concatenate([conv_1x1,conv_3x3,conv_5x5,max_pool], axis=-1)
    return out
def dense_block(x, filters=32, num_of_dense_block=3):
    for _ in range(num_of_dense_block):
        out = broadcast_block(x, filters)
        x = concatenate([x, out], axis=-1)
    return x
def transition_block(x):
    x = Conv2D(filters=32, kernel_size=1, strides=1, padding='same', kernel_initializer="lecun_normal")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=2, strides=2)(x)
    return x
def BBNN():
    input_layer = Input((img_width, img_height, 1))
    x = Conv2D(filters=32, kernel_size=3, strides=1, padding='same', kernel_initializer="lecun_normal")(input_layer)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=(4,1))(x)
    
    x = dense_block(x, filters=32, num_of_dense_block=4)
    x = transition_block(x)
    
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = GlobalAveragePooling2D()(x)
    output = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=input_layer, outputs=output)
    return model
model = BBNN()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 646, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 646, 128, 32  320         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 646, 128, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [5]:
%%timeit -n1 -r1
#without data argumatent
k_fold=0
train_size = 0.8
val_size = 0.1
test_size = 0.1

test_acc = 0


epochs = 100
batch_size = 8
lr = 0.01
file_name0 = 'Ballroom_our_TRAIN_UNTIL'
path  = './logs/Ballroom_our_TRAIN_UNTIL/'
csv_name0 = 'Ballroom_csv.csv'
train_loss_record = []
train_acc_record = []
val_loss_record = []
val_acc_record = []
test_loss_record = []
test_acc_record = []
while(test_acc < 0.615):
    print('Start %d fold training' % (k_fold+1))
    X_train, y_train, X_val, y_val, X_test, y_test = train_val_test_split(X_melspec, y, train_size=train_size, 
                                                                          val_size=val_size, test_size=test_size)
    file_name = path+str(k_fold)+'_fold_'+file_name0
    csv_path  = path+str(k_fold)+'_fold_'+ csv_name0
    lr_change = ReduceLROnPlateau(monitor="loss", factor=0.5, patience=3, min_lr=0.000)
    model_checkpoint = ModelCheckpoint(file_name, monitor='val_accuracy', save_best_only=True, mode='max')
    early_stopping = EarlyStopping(monitor='loss', min_delta=0.01, patience=10, mode='min')
    csv_logger = CSVLogger(csv_path)
    callbacks =[lr_change, model_checkpoint, early_stopping,csv_logger]
    opt = Adam(learning_rate=lr)
    model = BBNN()
    model.compile(
                loss='categorical_crossentropy',
                metrics=['accuracy'],
                optimizer=opt)
    hist = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, 
              validation_data=(X_val, y_val), verbose=1,
              callbacks=callbacks)
    model_best = load_model(file_name)
    train_loss, train_acc = model_best.evaluate(X_train, y_train, verbose=0)
    val_loss, val_acc = model_best.evaluate(X_val, y_val, verbose=0)
    test_loss, test_acc = model_best.evaluate(X_test, y_test, verbose=0)
    
    train_loss_record.append(train_loss)
    train_acc_record.append(train_acc)
    val_loss_record.append(val_loss)
    val_acc_record.append(val_acc)
    test_loss_record.append(test_loss)
    test_acc_record.append(test_acc)
    print('\n\n%d fold train loss %.4f train acc %.4f, val loss %.4f val acc %.4f, test loss %.4f test acc %.4f\n\n' % 
          (i+1, train_loss, train_acc, val_loss, val_acc, test_loss, test_acc))
    
    # Confusion Matrix code
    y_pred = model_best.predict(X_test)
    cf_matrix = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))
    np.save(path+f"{k_fold}_cf_matrix.npy", np.array(cf_matrix))
    
    k_fold += 1
    
# Saving the train_val_test record
np.save(path+"train_loss.npy", np.array(train_loss_record))
np.save(path+"train_acc.npy", np.array(train_acc_record))
np.save(path+"val_loss.npy", np.array(val_loss_record))
np.save(path+"val_acc.npy", np.array(val_acc_record))
np.save(path+"test_loss.npy", np.array(test_loss_record))
np.save(path+"test_acc.npy", np.array(test_acc_record))

# Getting the average data
train_loss_avg = np.mean(np.array(train_loss_record))
train_acc_avg = np.mean(np.array(train_acc_record))
val_loss_avg = np.mean(np.array(val_loss_record))
val_acc_avg = np.mean(np.array(val_acc_record))
test_loss_avg = np.mean(np.array(test_loss_record))
test_acc_avg = np.mean(np.array(test_acc_record))
print('\n\n%d fold train loss avg %.4f train acc avg %.4f, val loss avg %.4f val acc avg %.4f, test loss avg %.4f test acc avg %.4f' % 
  (k_fold, train_loss_avg, train_acc_avg, val_loss_avg, val_acc_avg, test_loss_avg, test_acc_avg))

Start 1 fold training
Epoch 1/100
100/100 [==============================] - 34s 289ms/step - loss: 1.8776 - accuracy: 0.3383 - val_loss: 5.4968 - val_accuracy: 0.2200 - lr: 0.0100
Epoch 2/100
100/100 [==============================] - 27s 273ms/step - loss: 1.6501 - accuracy: 0.3947 - val_loss: 3.4414 - val_accuracy: 0.3400 - lr: 0.0100
Epoch 3/100
100/100 [==============================] - 20s 201ms/step - loss: 1.5332 - accuracy: 0.4348 - val_loss: 4.0010 - val_accuracy: 0.2800 - lr: 0.0100
Epoch 4/100
100/100 [==============================] - 27s 272ms/step - loss: 1.4461 - accuracy: 0.5025 - val_loss: 1.6160 - val_accuracy: 0.4800 - lr: 0.0100
Epoch 5/100
100/100 [==============================] - 20s 201ms/step - loss: 1.3778 - accuracy: 0.5326 - val_loss: 2.7239 - val_accuracy: 0.3900 - lr: 0.0100
Epoch 6/100
100/100 [==============================] - 20s 201ms/step - loss: 1.2636 - accuracy: 0.5476 - val_loss: 1.9305 - val_accuracy: 0.4100 - lr: 0.0100
Epoch 7/100
100/100 [===

100/100 [==============================] - 20s 200ms/step - loss: 0.1515 - accuracy: 0.9624 - val_loss: 0.8893 - val_accuracy: 0.8200 - lr: 3.1250e-04
Epoch 85/100
100/100 [==============================] - 20s 200ms/step - loss: 0.1572 - accuracy: 0.9612 - val_loss: 0.8943 - val_accuracy: 0.8200 - lr: 1.5625e-04
Epoch 86/100
100/100 [==============================] - 20s 200ms/step - loss: 0.1615 - accuracy: 0.9586 - val_loss: 0.8808 - val_accuracy: 0.8100 - lr: 1.5625e-04
Epoch 87/100
100/100 [==============================] - 20s 200ms/step - loss: 0.1665 - accuracy: 0.9536 - val_loss: 0.8827 - val_accuracy: 0.8100 - lr: 1.5625e-04
Epoch 88/100
100/100 [==============================] - 20s 200ms/step - loss: 0.1572 - accuracy: 0.9612 - val_loss: 0.8847 - val_accuracy: 0.8000 - lr: 7.8125e-05
Epoch 89/100
100/100 [==============================] - 20s 200ms/step - loss: 0.1525 - accuracy: 0.9649 - val_loss: 0.8963 - val_accuracy: 0.8000 - lr: 7.8125e-05
Epoch 90/100
100/100 [=======

100/100 [==============================] - 27s 273ms/step - loss: 0.5996 - accuracy: 0.8008 - val_loss: 0.9946 - val_accuracy: 0.6300 - lr: 0.0100
Epoch 34/100
100/100 [==============================] - 20s 201ms/step - loss: 0.6056 - accuracy: 0.7920 - val_loss: 1.6435 - val_accuracy: 0.5700 - lr: 0.0100
Epoch 35/100
100/100 [==============================] - 20s 200ms/step - loss: 0.5897 - accuracy: 0.7970 - val_loss: 1.1643 - val_accuracy: 0.6200 - lr: 0.0100
Epoch 36/100
100/100 [==============================] - 27s 272ms/step - loss: 0.5669 - accuracy: 0.8033 - val_loss: 1.2433 - val_accuracy: 0.6500 - lr: 0.0100
Epoch 37/100
100/100 [==============================] - 20s 201ms/step - loss: 0.6310 - accuracy: 0.7857 - val_loss: 4.2548 - val_accuracy: 0.3300 - lr: 0.0100
Epoch 38/100
100/100 [==============================] - 27s 272ms/step - loss: 0.5427 - accuracy: 0.8008 - val_loss: 0.7850 - val_accuracy: 0.7200 - lr: 0.0100
Epoch 39/100
100/100 [==============================]

Epoch 17/100
100/100 [==============================] - 20s 201ms/step - loss: 0.9109 - accuracy: 0.6967 - val_loss: 2.0142 - val_accuracy: 0.5200 - lr: 0.0100
Epoch 18/100
100/100 [==============================] - 20s 201ms/step - loss: 0.8482 - accuracy: 0.7018 - val_loss: 1.4077 - val_accuracy: 0.5500 - lr: 0.0100
Epoch 19/100
100/100 [==============================] - 20s 201ms/step - loss: 0.8122 - accuracy: 0.7256 - val_loss: 0.9827 - val_accuracy: 0.6700 - lr: 0.0100
Epoch 20/100
100/100 [==============================] - 20s 200ms/step - loss: 0.7461 - accuracy: 0.7544 - val_loss: 1.5483 - val_accuracy: 0.4700 - lr: 0.0100
Epoch 21/100
100/100 [==============================] - 20s 201ms/step - loss: 0.7786 - accuracy: 0.7393 - val_loss: 1.9845 - val_accuracy: 0.5400 - lr: 0.0100
Epoch 22/100
100/100 [==============================] - 20s 201ms/step - loss: 0.7770 - accuracy: 0.7381 - val_loss: 1.6480 - val_accuracy: 0.6000 - lr: 0.0100
Epoch 23/100
100/100 [==================

Epoch 65/100
100/100 [==============================] - 20s 201ms/step - loss: 0.1463 - accuracy: 0.9749 - val_loss: 0.6129 - val_accuracy: 0.8200 - lr: 0.0012
Epoch 66/100
100/100 [==============================] - 20s 201ms/step - loss: 0.1466 - accuracy: 0.9687 - val_loss: 0.5718 - val_accuracy: 0.7800 - lr: 0.0012
Epoch 67/100
100/100 [==============================] - 20s 200ms/step - loss: 0.1610 - accuracy: 0.9511 - val_loss: 0.6025 - val_accuracy: 0.7900 - lr: 0.0012
Epoch 68/100
100/100 [==============================] - 20s 201ms/step - loss: 0.1371 - accuracy: 0.9674 - val_loss: 0.5518 - val_accuracy: 0.8200 - lr: 6.2500e-04
Epoch 69/100
100/100 [==============================] - 20s 200ms/step - loss: 0.1300 - accuracy: 0.9699 - val_loss: 0.5105 - val_accuracy: 0.8100 - lr: 6.2500e-04
Epoch 70/100
100/100 [==============================] - 20s 200ms/step - loss: 0.1407 - accuracy: 0.9649 - val_loss: 0.5212 - val_accuracy: 0.8100 - lr: 6.2500e-04
Epoch 71/100
100/100 [======

Epoch 61/100
100/100 [==============================] - 20s 201ms/step - loss: 0.3091 - accuracy: 0.9148 - val_loss: 0.5036 - val_accuracy: 0.8300 - lr: 1.5625e-04
Epoch 62/100
100/100 [==============================] - 20s 200ms/step - loss: 0.2916 - accuracy: 0.9211 - val_loss: 0.4978 - val_accuracy: 0.8300 - lr: 7.8125e-05
Epoch 63/100
100/100 [==============================] - 20s 201ms/step - loss: 0.2944 - accuracy: 0.9173 - val_loss: 0.5041 - val_accuracy: 0.8200 - lr: 7.8125e-05
Epoch 64/100
100/100 [==============================] - 20s 201ms/step - loss: 0.2999 - accuracy: 0.9185 - val_loss: 0.5022 - val_accuracy: 0.8200 - lr: 7.8125e-05
Epoch 65/100
100/100 [==============================] - 20s 200ms/step - loss: 0.2934 - accuracy: 0.9236 - val_loss: 0.5068 - val_accuracy: 0.8200 - lr: 3.9062e-05
Epoch 66/100
100/100 [==============================] - 20s 201ms/step - loss: 0.2968 - accuracy: 0.9160 - val_loss: 0.5012 - val_accuracy: 0.8200 - lr: 3.9062e-05
Epoch 67/100
100